In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning) # setting ignore as a parameter and further adding category
from whispr import *
import os


In [6]:
# gene expression. 2.5ul rxns, 75% txtl
rxn_vol = 3
source_plate_type = '384PP_AQ_BP' 

os.getcwd()
folder = os.getcwd() + '/Experiments/TODO_mod2/'
sp_plasmids_file = folder + 'plasmids_sp.xlsx'
sp_plasmids = pd.read_excel(sp_plasmids_file, index_col = 0, engine='openpyxl')
sp_plasmids = sp_plasmids[~sp_plasmids['Well'].isna()]

layout_genex_file = folder + 'genex-pl-384.csv'
layout_genex = pd.read_csv(layout_genex_file, index_col = 0, dtype = str)

mt_genex_file = folder + 'genex-mt.xlsx'
#mt_genex = pd.read_csv(mt_genex_file, index_col = 0, dtype = str).fillna(0)
mt_genex = pd.read_excel(mt_genex_file, index_col = 0, dtype = str, engine = 'openpyxl').fillna(0)

checkInputs(sp_plasmids, mt_genex,source_plate_type)

vol_table_df = generateVolumeTable(mt_genex, sp_plasmids, rxn_vol = 0.25*rxn_vol, total_vol = rxn_vol, fill_with='water', multiRpW = False)

#specify rxn_vol (default = 2.5) and total_vol (default = 10) if you'd like to change the volume of each individual replicate or the total reaction volume
protocol_genex_df = writeProtocol(source_plate_type, vol_table_df, layout_genex,sp_plasmids, update_source_vol=folder+'230207_plasmids_sp_updated.xlsx')
protocol_genex_df = protocol_genex_df[0]


Volumes used from each well for this protocol:
{'1_A1': 1.525, '1_P3': 0.15, '1_P9': 0.15, '1_O1': 0.075, '1_O3': 0.075, '1_O5': 0.1, '1_O6': 0.1, '1_O7': 0.75, '1_O8': 0.1, '1_O9': 0.05, '1_O13': 0.9, '1_O14': 0.375, '1_O16': 0.15}


In [8]:
vol_table_df['ftl']

0    0.025
1        0
2    0.025
3        0
4    0.025
5        0
Name: ftl, dtype: object

In [6]:
# everything gets txtl

# get destination wells
for layout_id, layout in enumerate([layout_genex]):    
    # reads plate layout and assigns wells to each reaction (rxn_loc; dict)
    labels = pd.unique(np.concatenate(layout.values))
    rxn_loc = {}
    for l in labels:
        if type(l) is str:
            index = layout[layout.isin([l])].stack().index
            rxn_loc[l] = []
            for i in index:
                rxn_loc[l].append(str(i[0]) + str(i[1]))


txtl_well = 'B21'
for label, dwell in rxn_loc.items():
    for d in dwell:
        print(label)
        txtl_vol = vol_table_df[vol_table_df['Label'] == label].iloc[0,1:].sum()/0.25*0.75*1000 # add txtl to make 75%
        row = {'Source Plate Name':'Source[1]', 'Source Plate Type': source_plate_type, 'Source Well': txtl_well,
                                    'Destination Plate Name':'Destination[1]', 'Destination Well': d, 'Transfer Volume': txtl_vol}
        protocol_genex_df = protocol_genex_df.append(row, ignore_index = True)

protocol_genex_df.to_csv(folder+'protocol_genex.csv',index = False)
print('TXTL needed: '+str(protocol_genex_df[protocol_genex_df['Source Well'] == txtl_well]['Transfer Volume'].sum()/1000))
# carry out gene expression reaction

full_complex_mod1_lplA
full_complex_lplA
full_complex_mod1
full_complex
Hprot_mod1
Hprot
TXTL needed: 13.500000000000002


In [5]:
# dilution. everything gets Tris, for 20x -> 57uL

##### doing it by hand if not too many reactions #####

# protocol_dilution_df = pd.DataFrame(columns=protocol_genex_df.columns)
# source_plate_type = '6RES_AQ_B' 
# tris_well = 'A1'
# tris_vol = 57*1000

# for dwell in protocol_genex_df['Destination Well'].unique():
#     row = {'Source Plate Name':'Source[1]', 'Source Plate Type': source_plate_type, 'Source Well': tris_well,
#                                 'Destination Plate Name':'Destination[1]', 'Destination Well': dwell, 'Transfer Volume': tris_vol}
#     protocol_dilution_df = protocol_dilution_df.append(row, ignore_index = True)

# protocol_dilution_df.to_csv('protocols/221024_arpae_dilutions.csv',index = False)

In [6]:
# from source plate with buffer, mix into 96

# from source plate with genex, mix the diluted txtl

In [2]:
folder = os.getcwd() + '/Experiments/TODO_mod2/'


In [3]:
# biosynthesis. 25ul rxns, 2.5ul of diluted txtl

# buffer source plate
sp_buffers_file = folder + 'buffers_sp.xlsx'
sp_buffers= pd.read_excel(sp_buffers_file, index_col = 0, engine = 'openpyxl')
sp_buffers = sp_buffers[~sp_buffers['Well'].isna()]

# HEPES reservoir (actually same plate as Tris)
sp_hepes = pd.DataFrame(columns=sp_buffers.columns)
sp_hepes = sp_hepes.append({'Label':'HEPES', 'Well':'A2,A3', 'Concentration':'', 'Volume':'2000,2000'}, ignore_index = True)

# gene expression source plate (diluted w/ Tris)
#sp_genex = sp_from_layout(layout_genex, 60)
layout_genex_384_file = folder + 'genex-pl-384.csv'
layout_genex_384 = pd.read_csv(layout_genex_384_file, index_col = 0, dtype = str)
sp_genex = sp_from_layout(layout_genex_384, 60)


# group source plates
sp_types = ['384PP_AQ_BP','6RES_AQ_BP2','384PP_AQ_BP'] # triple check this
sps = [sp_buffers,sp_hepes,sp_genex]

# get mixing table
mt_biosyn_file = folder + 'buffers-mt.xlsx'
mt_biosyn = pd.read_excel(mt_biosyn_file, index_col = 0, dtype = str, engine = 'openpyxl').fillna(0)
mt_biosyn = mt_biosyn[~mt_biosyn.index.isna()]

# check formats
checkInputs(sps,mt_biosyn,sp_types)

# layouts for destination plate(s)
layout_biosyn1_file = folder + 'biosyn_pl.csv' 
layout_biosyn1 = pd.read_csv(layout_biosyn1_file, index_col = 0, dtype = str)
layout_biosyn1 = layout_biosyn1[~layout_biosyn1.index.isna()]

# filename = '221018_arpae_biosyn_neg.csv'
# layout_biosyn2_file = 'plate_layouts/'+filename 
# layout_biosyn2 = pd.read_csv(layout_biosyn2_file, index_col = 0, dtype = str)
# layouts = [layout_biosyn1,layout_biosyn2]

vol_table_df = generateVolumeTable(mt_biosyn, sps, rxn_vol = 22.5, total_vol = 25, fill_with='HEPES')

protocol_biosyn_dfs = writeProtocol(sp_types, vol_table_df, layout_biosyn1,sps, update_source_vol= folder +'combined_sp_updated.xlsx')

#protocol_biosyn_df.to_csv('protocols/221018_arpae_biosyn.csv',index = False)

<class 'int'>


AttributeError: 'int' object has no attribute 'sort_values'

In [7]:
sps[0].loc[['ATP']].sort_values(by = 'Concentration')['Concentration'].sort_values()

Item
ATP    50
Name: Concentration, dtype: object

In [15]:
protocol_biosyn_dfs[0].to_csv(folder+'biosyn_protocol_384.csv',index = False)
protocol_biosyn_dfs[1].to_csv(folder+'biosyn_protocol_6RES.csv',index = False)

In [71]:
missing = []
for m in mt_biosyn.columns.str.casefold():
    missing.append(m in sp_buffers['Label'].str.casefold().values)
false_val = [not value for value in missing]
mt_biosyn.columns[false_val]

Index(['ATP', 'full_complex_mod1_lplA', 'full_complex_lplA',
       'full_complex_mod1', 'full_complex', 'Hprot_mod1', 'Hprot'],
      dtype='object')

In [6]:
mt_biosyn

,formate,THF,"5,10-CH2-THF",NADH,ATP,lipoic_acid,FAD,PLP,Mg-Act,NH4OH,HCO3,Kan,KOH,DTT,full_complex_mod1_lplA,full_complex_lplA,full_complex_mod1,full_complex,Hprot_mod1,Hprot
full_complex_mod1_lplA,15,5,0,1,1,1,0.25,0.25,1,1,1,1,30,0,10,0,0,0,0,0
full_complex_lplA,15,0,5,1,0,1,0.25,0.25,0,1,1,1,30,0,0,10,0,0,0,0
full_complex_mod1,15,5,0,1,1,1,0.25,0.25,1,1,1,1,30,0,0,0,10,0,0,0
full_complex,15,0,5,1,0,1,0.25,0.25,0,1,1,1,30,0,0,0,0,10,0,0
Hprot_mod1,15,5,0,1,1,1,0.25,0.25,1,1,1,1,30,0,0,0,0,0,10,0
Hprot,15,0,5,1,0,1,0.25,0.25,0,1,1,1,30,0,0,0,0,0,0,10
full_complex_mod1_lplA_DTT,15,5,0,1,1,1,0.25,0.25,1,1,1,1,30,5,10,0,0,0,0,0
full_complex_lplA_DTT,15,0,5,1,0,1,0.25,0.25,0,1,1,1,30,5,0,10,0,0,0,0
full_complex_mod1_DTT,15,5,0,1,1,1,0.25,0.25,1,1,1,1,30,5,0,0,10,0,0,0
full_complex_DTT,15,0,5,1,0,1,0.25,0.25,0,1,1,1,30,5,0,0,0,10,0,0
